In [6]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

#TODO: Mejorar grafica, revisar la magnitud de las ventanas (1 ventana de 1 hora =/= 1 ventana de 5 horas)

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def analyze_room_compaction(room_data):
    """Analiza la compactación del horario de una sala."""
    schedule = [[False for _ in range(9)] for _ in range(5)]
    total_classes = len(room_data['Asignaturas'])
    
    # Marcar bloques ocupados
    for class_data in room_data['Asignaturas']:
        days = {'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4}
        day_idx = days.get(class_data['Dia'], -1)
        if day_idx != -1:
            block_idx = class_data['Bloque'] - 1
            if 0 <= block_idx < 9:
                schedule[day_idx][block_idx] = True

    # Analizar ventanas por día
    total_windows = 0
    total_duration = 0
    
    for day_schedule in schedule:
        first_occupied = -1
        last_occupied = -1
        for i, block in enumerate(day_schedule):
            if block:
                if first_occupied == -1:
                    first_occupied = i
                last_occupied = i
        
        if first_occupied != -1:
            in_window = False
            current_window_length = 0
            for i in range(first_occupied, last_occupied + 1):
                if not day_schedule[i]:
                    current_window_length += 1
                    in_window = True
                else:
                    if in_window:
                        total_windows += 1
                        total_duration += current_window_length
                        current_window_length = 0
                        in_window = False
    
    return {
        'total_windows': total_windows,
        'total_duration': total_duration,
        'total_classes': total_classes
    }

def create_summary_dataframe(horarios_salas):
    """Crea un DataFrame con los datos de compactación."""
    summary_data = []
    
    for sala in horarios_salas:
        stats = analyze_room_compaction(sala)
        summary_data.append({
            'Codigo': sala['Codigo'],
            'Ventanas': stats['total_windows'],
            'Duracion': stats['total_duration'],
            'Clases': stats['total_classes']
        })
    
    return pd.DataFrame(summary_data)

def plot_bar_chart(data):
    # Ordenar por cantidad de ventanas + duración para mejor visualización
    data['Total'] = data['Cantidad de Ventanas'] + data['Duracion de Ventanas'] 
    data_sorted = data.sort_values('Total', ascending=True)
    data_sorted = data_sorted.drop('Total', axis=1)
    
    # Configurar el gráfico
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Posiciones de las barras
    x = np.arange(len(data_sorted))
    width = 0.35
    
    # Crear las barras
    bars1 = ax.bar(x - width/2, data_sorted['Cantidad de Ventanas'], 
                  width, label='Cantidad de Ventanas', color='#2ecc71')
    bars2 = ax.bar(x + width/2, data_sorted['Duracion de Ventanas'], 
                  width, label='Duración de Ventanas', color='#3498db')
    
    # Personalizar el gráfico
    ax.set_ylabel('Número')
    ax.set_title('Cantidad y Duración de Ventanas por Sala')
    ax.set_xticks(x)
    ax.set_xticklabels(data_sorted['Codigo'], rotation=45, ha='right')
    ax.legend()
    
    # Añadir cuadrícula horizontal para facilitar la lectura
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)
    
    # Ajustar los márgenes para que las etiquetas sean visibles
    plt.tight_layout()
    
    # Guardar el gráfico
    plt.savefig('ventanas_barras.png')
    plt.close()

def main():
    # Cargar datos
    try:
        with open('../../agent_output/Horarios_salas.json', 'r', encoding='utf-8') as file:
            horarios_salas = json.load(file)
    except Exception as e:
        print(f"Error al cargar el archivo: {str(e)}")
        return
    
    try:
        df = pd.read_csv('resumen_compactacion.csv')
        # Eliminar la fila 'Total' si existe
        df = df[df['Codigo'] != 'Total']
        plot_bar_chart(df)
        print("Gráfico generado exitosamente")
    except Exception as e:
        print(f"Error al generar el gráfico: {str(e)}")
    
    print("\nVisualizaciones generadas exitosamente")

if __name__ == "__main__":
    main()

Gráfico generado exitosamente

Visualizaciones generadas exitosamente
